# **Data Exploration for Process Mining**

In [ ]:
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# setting some options:
# pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

cwd = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(cwd))
sys.path.append(root_dir)
sys.path.append(root_dir)

from src.loader import TripLoader

trip_loader = TripLoader()
trips_data_ABCD = trip_loader.trips_ABCD
trips_data_MNOP = trip_loader.trips_MNOP
trips_data_ZYXW = trip_loader.trips_ZYXW

In [ ]:
# new flight id

trips_data = trips_data_ABCD
trips_data["flight_id"] = (
    trips_data["airline_code"].astype(str)
    + "_"
    + trips_data["flight_number"].astype(str)
    + "_"
    + trips_data["flight_date"].astype(str)
    + "_"
    + trips_data["departure_airport"].astype(str)
)
overview = [
    "id",
    "creation_time",
    "action_name",
    "flight_id",
    "airline_code",
    "flight_number",
    "flight_date",
    "departure_airport",
]
trips_data[overview].head(3)

In [ ]:
trips_data[overview].sort_values(by="flight_id")

In [ ]:
trips_data.columns

In [ ]:
trips_data["action_name"].unique()

In [ ]:
trips_data[
    (trips_data["flight_id"] == "AB_1070_15_BOM")
    & (trips_data["header_category"] == "received")
][overview]

**creation_time exploration**<br>
Es scheint, dass alle bzw. sehr viele Einträge eines Fluges die gleiche `creation_time` haben. Dies würde ein Process Mining erschweren bzw. unmöglich machen, da durch den Zeitstempel die Zusammenhänge und Reihenfolgen erkannt werden können. Im Folgenden wird diese Beobachtung genauer untersucht.

Zum gruppieren der Flüge wird die oben erstellte `flight_id` genutzt, welche aus `code, number, date und departure airport` besteht.
Da Einträge zudem doppelt sind sollten die Flüge nach der  `header_category` saved oder received gefiltert werden.

Der aktuelle Zeitstempel enthält keine Millisekunden, diese werden aus `header_line` extrahiert und der Zeitstempel geupdated.

In [ ]:
import pandas as pd


def explore_times(trips_data, filter_option="received", aggregation_method="header_id"):
    """
    Aggregates flight data based on the specified filter option and aggregation method.

    Parameters:
    trips_data (pd.DataFrame): The DataFrame containing flight data.
    filter_option (str): The filter for 'header_category', either 'received' or 'saved'. Default is 'received'.
    aggregation_method (str): The aggregation method, either 'header_id' or 'flight_id'. Default is 'header_id'.

    Returns:
    pd.DataFrame: The aggregated result.
    """
    # Filter the DataFrame by 'header_category'
    filtered_data = trips_data[trips_data["header_category"] == filter_option]

    # Convert 'creation_time' to datetime
    filtered_data["creation_time"] = pd.to_datetime(filtered_data["creation_time"])

    # Extract milliseconds from 'header_line'
    filtered_data["milliseconds"] = filtered_data["header_line"].apply(
        lambda x: int(x.split(",")[1].split()[0])
    )

    # Include milliseconds in 'creation_time'
    filtered_data["creation_time"] = filtered_data.apply(
        lambda row: row["creation_time"]
        + pd.Timedelta(milliseconds=row["milliseconds"]),
        axis=1,
    )

    # Set the group_column based on the aggregation_method
    group_column = "flight_id" if aggregation_method == "flight_id" else "header_id"

    # Group by the chosen column and aggregate
    result = (
        filtered_data.groupby(group_column)
        .agg(
            num_entries=(group_column, "size"),
            all_time_same=("creation_time", lambda x: x.nunique() == 1),
            num_unique_time=("creation_time", "nunique"),
            creation_times=("creation_time", lambda x: list(x)),
        )
        .reset_index()
    )

    # Calculate the number of duplicates in creation_times
    result["num_duplicates"] = result["creation_times"].apply(
        lambda x: len(x) - len(set(x))
    )

    # Reorder columns for better readability
    result = result[
        [
            group_column,
            "num_entries",
            "all_time_same",
            "num_unique_time",
            "creation_times",
            "num_duplicates",
        ]
    ]

    # Rename group_column to 'aggregation_id' for consistency
    result = result.rename(columns={group_column: "aggregation_id"})

    return result

In [ ]:
result = explore_times(
    trips_data, filter_option="received", aggregation_method="flight_id"
)
result

In [ ]:
false_count = (result["all_time_same"] == False).sum()
print(
    f"Anzahl der Flüge ohne ausschließlich gleiche Zeitstempeln: {false_count} von {result.shape[0]} ({false_count/result.shape[0]*100:.2f}%)"
)

**Betrachtung wie die Zeiten der Aktivitäten verteilt sind**

In [ ]:
# Flug AB_1070_15_BOM
times = result[result["aggregation_id"] == "AB_1070_15_BOM"]["creation_times"].iloc[0]
times

In [ ]:
# Calculate the time difference between the max and min timestamps in seconds
result["time_difference_seconds"] = result["creation_times"].apply(
    lambda x: (max(x) - min(x)).total_seconds()
)

# Data for the plot
data = result["time_difference_seconds"]

# Create a figure and a set of subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={"width_ratios": [3, 1]})

# Histogram on the first subplot
axs[0].hist(data, bins=150, color="blue", edgecolor="black")
axs[0].set_title("Distribution of Time Differences (Seconds)")
axs[0].set_xlabel("Time Difference (seconds)")
axs[0].set_ylabel("Frequency")
axs[0].set_xlim(
    left=0, right=max(data) + 100
)  # Adjust the upper limit based on your data

# Boxplot on the second subplot
axs[1].boxplot(data, vert=True)  # 'vert=True' makes the boxplot vertical
axs[1].set_title("Box Plot of Time Differences")
axs[1].set_ylabel("Time Difference (seconds)")

# Adjust layout to prevent overlapping
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
result["time_difference_seconds"] = result["creation_times"].apply(
    lambda x: (max(x) - min(x)).total_seconds()
)
print(result["time_difference_seconds"].describe())

In [ ]:
print(147182.993662 / 60 / 60)
print(589499.025000 / 60 / 60)

In [ ]:
# Define the range for filtering
lower_bound = 1
upper_bound = 60 * 60

# Filter the data based on the specified range
filtered_data = result[
    result["time_difference_seconds"].between(lower_bound, upper_bound)
]["time_difference_seconds"]

# Create a figure and a set of subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={"width_ratios": [3, 1]})

# Histogram on the first subplot for filtered data
axs[0].hist(filtered_data, bins=50, color="green", edgecolor="black")
axs[0].set_title(
    f"Distribution of Time Differences (Seconds) \nRange: {lower_bound}-{upper_bound} seconds"
)
axs[0].set_xlabel("Time Difference (seconds)")
axs[0].set_ylabel("Frequency")
axs[0].set_xlim(left=lower_bound, right=upper_bound)

# Boxplot on the second subplot for filtered data
axs[1].boxplot(filtered_data, vert=True)  # 'vert=True' makes the boxplot vertical
axs[1].set_title("Box Plot of Time Differences")
axs[1].set_ylabel("Time Difference (seconds)")

# Adjust layout to prevent overlapping
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
filtered_data = result[
    result["time_difference_seconds"].between(lower_bound, upper_bound)
]["time_difference_seconds"]
filtered_data.describe()

In [ ]:
# Define the range for filtering
lower_bound = 77650
upper_bound = 77750  # 60*60

# Filter the data based on the specified range
filtered_data = result[
    result["time_difference_seconds"].between(lower_bound, upper_bound)
]["time_difference_seconds"]

# Create a figure and a set of subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={"width_ratios": [3, 1]})

# Histogram on the first subplot for filtered data
axs[0].hist(filtered_data, bins=50, color="green", edgecolor="black")
axs[0].set_title(
    f"Distribution of Time Differences (Seconds) \nRange: {lower_bound}-{upper_bound} seconds"
)
axs[0].set_xlabel("Time Difference (seconds)")
axs[0].set_ylabel("Frequency")
axs[0].set_xlim(left=lower_bound, right=upper_bound)

# Boxplot on the second subplot for filtered data
axs[1].boxplot(filtered_data, vert=True)  # 'vert=True' makes the boxplot vertical
axs[1].set_title("Box Plot of Time Differences")
axs[1].set_ylabel("Time Difference (seconds)")

# Adjust layout to prevent overlapping
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
filtered_data = result[
    result["time_difference_seconds"].between(lower_bound, upper_bound)
]["time_difference_seconds"]
filtered_data.describe()

In [ ]:
print(77697.196005 / 60 / 60)

Betrachtung Flug mit maximal vergangener Zeit:

In [ ]:
# Find the row with the maximum time difference
max_diff_row = result[
    result["time_difference_seconds"] == result["time_difference_seconds"].max()
]
max_diff_row

In [ ]:
filtered_data = trips_data[
    (trips_data["flight_id"] == "AB_2128_7_DEL")
    & (trips_data["header_category"] == "received")
]
filtered_data[overview]

In [ ]:
filtered_data["action_name"].value_counts()

Betrachtung Flug mit minimal vergangener Zeit:

In [ ]:
# Find the row with the minimum time difference
min_diff_row = result[
    result["time_difference_seconds"] == result["time_difference_seconds"].min()
]
min_diff_row.sort_values(by="num_entries", ascending=False).head(3)

In [ ]:
trips_data[
    (trips_data["flight_id"] == "AB_1070_17_BOM")
    & (trips_data["header_category"] == "received")
][overview]

Betrachtung der Flüge die unterschiedliche Zeitstempel haben:

In [ ]:
false_entries = result[result["all_time_same"] == False]
false_aggregation_ids = result[result["all_time_same"] == False]["aggregation_id"]
false_entries["portion_unique_times"] = (
    false_entries["num_unique_time"] / false_entries["num_entries"]
)
false_entries = false_entries[
    [
        "aggregation_id",
        "num_entries",
        "all_time_same",
        "num_unique_time",
        "portion_unique_times",
        "creation_times",
        "num_duplicates",
    ]
]

false_entries.head()

In [ ]:
average_portion = false_entries["portion_unique_times"].mean()
print("Durchschnittlicher Anteil der unique_times:", average_portion)

**Betrachtung der Anzahl an Aktivitäten die für einen Flug durchgeführt wurden**

In [ ]:
print(result["num_entries"].describe())

In [ ]:
# Assuming result['num_entries'] is your data array
data = result["num_entries"]

# Create a figure and a set of subplots
fig, axs = plt.subplots(
    1, 2, figsize=(12, 5), gridspec_kw={"width_ratios": [3, 1]}
)  # 1 row, 2 columns, figure size of 12x5 inches

# Histogram on the first subplot
axs[0].hist(data, bins=150, color="blue", edgecolor="black")
axs[0].set_title("Distribution of num_entries")
axs[0].set_xlabel("Number of Entries")
axs[0].set_ylabel("Frequency")
axs[0].set_xlim(left=0, right=450)

# Boxplot on the second subplot
axs[1].boxplot(data, vert=True)  # 'vert=True' makes the boxplot vertical
axs[1].set_title("Box Plot of num_entries")
axs[1].set_ylabel("Number of Entries")

# Adjust layout to prevent overlapping
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
filtered_trips_data = trips_data[trips_data["flight_id"].isin(false_aggregation_ids)]
action_name_counts = filtered_trips_data[overview]["action_name"].value_counts()
action_name_counts

In [ ]:
# Plot the value counts
plt.figure(figsize=(10, 6))
action_name_counts.plot(kind="bar")
plt.title("Action Name Value Counts")
plt.xlabel("Action Name")
plt.ylabel("Count")
plt.xticks(rotation=90)
plt.show()

Es scheint teils Flüge zu geben die **sehr viele** Aktivitäten haben (>200).

**Learnings:**
- 93.24% der Flüge haben Aktivtäten mit unterschiedlichen Zeitstempeln
- Zeitunterschiede zwischen den Aktivitäten reichen von 0s bis 163h
- Verteilung der Zeitunterschiede sehr breit mit teils starken Spikes
- Anzahl der Aktivitäten pro Flug variiert mit für die meinsten Flüge zwischen 0 und 50 sowie 250 und 400



**Process Mining**
Folgende Informationen werden benötigt:
- Case ID --> `flight_id`
- Activity --> `action_name` 
- Timestamp --> `creation_time` mit milisekunden

Optional
- Resource
- Activity Duration
- ?


**Testdaten extrahieren für Mining**

In [ ]:
trip_loader = TripLoader()
trips_data_ABCD = trip_loader.trips_ABCD
# trips_data_MNOP = trip_loader.trips_MNOP
# trips_data_ZYXW = trip_loader.trips_ZYXW
trips_data = trips_data_ABCD

# new flight id
trips_data = trips_data_ABCD
trips_data["flight_id"] = (
    trips_data["airline_code"].astype(str)
    + "_"
    + trips_data["flight_number"].astype(str)
    + "_"
    + trips_data["flight_date"].astype(str)
    + "_"
    + trips_data["departure_airport"].astype(str)
)

trips_data["creation_time"] = pd.to_datetime(trips_data["creation_time"])

# Extracting milliseconds from 'header_line'
trips_data["milliseconds"] = trips_data["header_line"].apply(
    lambda x: int(x.split(",")[1].split()[0])
)

# Include milliseconds in 'creation_time'
trips_data["creation_time"] = trips_data.apply(
    lambda row: row["creation_time"] + pd.Timedelta(milliseconds=row["milliseconds"]),
    axis=1,
)

mining_columns = ["id", "flight_id", "action_name", "creation_time"]

mining_data = trips_data[mining_columns]
mining_data

In [ ]:
mining_data.to_csv("data/24-06-09_trips_abcd_mining_01.csv", index=False)